<center>
<h1> Prueba DS - MELI </h1>
    <h2> Match de productos</h2>

## Búsqueda de Productos Similares

El siguiente ejercicio tiene como objetivo encontrar productos con mayor similitud a un producto específico. Para lograr esto, necesitamos el `MLA_id` del producto que nos interesa. Utilizaremos este identificador para encontrar el artículo que tenga la mayor similitud en la categoría correspondiente. Funciona únicamente para productos de ML Argentina.

### Pasos:

1. **Obtén el `MLA_id` del Producto de Interés:** Identifica el producto para el cual deseas encontrar productos similares. Anota el `MLA_id` que te interese de la base objetivo.

2. **Búsqueda de Similitud:** Utilizando el `MLA_id` del producto de interés, buscaremos el artículo más similar dentro de la misma categoría. Este proceso se basa en comparar características clave de los productos, como sus títulos, y encontrar aquellos que compartan patrones similares.

3. **Mejora de la Experiencia del Usuario:** Este enfoque tiene como objetivo mejorar la experiencia del usuario al ofrecer opciones similares al producto que están considerando. Al mostrar productos relacionados, aumentamos las posibilidades de que encuentren lo que están buscando de manera más eficiente.

En resumen, el objetivo de este ejercicio es ayudar a los usuarios a encontrar productos similares al que les interesa, mejorando su experiencia de compra y exploración en la plataforma.

Ejecutar las celdas en orden hasta llegar al markdown de "Solución"

In [96]:
import requests
import pandas as pd
import os
import json
import re
import pickle
import builtins
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from unidecode import unidecode
#Similitud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import scipy.stats

In [97]:
#Clase para generar insumos necesarios para análisis

class data_inicial():
#Funciones encargadas de generar la base sobre la cual vamos a trabajar problema de similitudes.
    
    def extrae_cat_objetivo(self,MLA_id):
        url = f'https://api.mercadolibre.com/items/{MLA_id}'
        request = requests.get(url)
        items = request.json()
        return [items['title'], items['category_id'],items['price']]
    

    #Funcion que ingresa en el item unico y extrae la foto en calidad no reducida != Thumbnail
    #Ideal realizar modelo de anpalisis de imagenes por eso se extrae link de imagen de buena calidad

    def get_image(self,id_sku):
        url = f'https://api.mercadolibre.com/items/{id_sku}'
        request = requests.get(url)
        item = request.json()
        # Limita a solo 1 foto, opcional más a futuro, pero será necesario un array y un loop para guardarlas
        pictures = item.get('pictures', [])[:1]  
        picture_url = pictures[0].get('secure_url', None) if pictures else None
        #Retorna string de la url
        return picture_url


    # Funcion para limpiar y estandarizar el titutlo deL producto
    def clean_text(self,text):
        #Se eliminan tildes/acentos y carpacteres especiales
        #No se eliminan numeros con el regex
        text = unidecode(text.lower())
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

        # Tokenizacion del titulo y lematizacion para llevar la palabras a terminos comunes
        words = word_tokenize(text, language='spanish')
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(word) for word in words]

        # Se eliminan stopwords en espanol
        stop_words = set(stopwords.words('spanish'))
        words = [word for word in words if word not in stop_words]

        cleaned_text = ' '.join(words)
        return cleaned_text
    
    #Función que retorna lista de titulos limpios
    def busca_titulos(self,dataframe)-> list[str]:
        return dataframe['cleaned_title'].tolist()

    #Función que crea el dataframe con las variables necesarias
    #Parámetros Category_id
    #Para temas del ejercicio se utiliza offset dentro de un loop para rescatar valores hasta 1000 items donde sea posible
    #the requested limit is higher than the allowed for public users. Maximum allowed is 50 -> error al usar parametro limit
    # si a futuro es necesario buscar más de 1000 registros se debe modificar parámetro en la API CON "search_type=scan"
    #necesario cuenta dev en MELI
    #https://developers.mercadolibre.com.co/es_ar/items-y-busquedas
    
    #cambiar target_results para tener más registros.
    
    def base_inicial(self,cat_id):
        target_results = 200
        results_per_page = 50  
        base_url = f'https://api.mercadolibre.com/sites/MLA/search?category={cat_id}'
        offset = 0
        filtered_items = []

        while len(filtered_items) < target_results:
            url = f'{base_url}&offset={offset}'
            request = requests.get(url)
            items = request.json()

            #Itera sobre cada una de las ramas del json de donde nos importa sacar la información necesaria (Attributes)
            for item in items['results']:
                attributes = item.get('attributes', [])
                filtered_item = {
                    'MLA_id': item['id'],
                    'title': item['title'],
                    'condition': item['condition'],
                    'permalink': item['permalink'],
                    'category_id': item['category_id'],
                    'domain_id': item['domain_id'],
                    'tags': item['tags'],
                    'price': item['price'],
                    'brand': next((attr['value_name'] for attr in attributes if attr['id'] == 'BRAND'), None),
                    'model': next((attr['value_name'] for attr in attributes if attr['id'] == 'MODEL'), None),
                    'thumbnail': item['thumbnail']
                }
                filtered_items.append(filtered_item)
                
            offset += results_per_page
            if len(items['results']) < results_per_page:
                break

        df_ini = pd.DataFrame(filtered_items)
        #aplica función de extraer la imagen a cada item del dataframe
        df_ini['url'] = df_ini.apply(lambda row: self.get_image(row['MLA_id']), axis=1)
        #aplica función de limpiar titulo de la publicacion
        df_ini['cleaned_title'] = df_ini.apply(lambda row: self.clean_text(row['title']), axis=1)
        #Resultado es un df con toda la información que se utilizará
        return df_ini

        

In [94]:
#Funciones para anális de similitud

#Genera modelo de tfdif
#Funcion para extraer los titulos limpios de la base objetivo
def genera_modelo_tfdif(df):
    #Se asegura de estar en la misma ubicación que el notebook
    notebook_location = os.path.dirname(os.path.abspath("__file__"))
    os.chdir(notebook_location)
    data = data_inicial()
    
    #lee titulos de productos limpios y este es el inusmo del modelo
    names = data.busca_titulos(df)
    #validaremos caracteres y combinaciones hasta de 5 n-gramas
    tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1, 5))
    tfidf.fit(names)
    
    #Sobreescribe modelo si ya existe previamente, pues va a cambiar con cada base que estemos buscando
    model_path = os.path.join("models", "names.pkl")
    try:
        with builtins.open(model_path, "wb") as fout:
            pickle.dump(tfidf, fout)
        print("New model created and saved successfully.")
    except Exception as e:
        print(f"Error saving the model: {e}")
    
    return tfidf

#Genera funcion para correr modelo tfid de similitud sobre los titulos
def similitud_nombre(titulo: str, nombres_base: list[str],df) -> dict[int, float]:
    #Genera modelo tfdif y calcula vector de similitud
    tfidf = genera_modelo_tfdif(df)
    titulo_vec = tfidf.transform([titulo])
    target_vec = tfidf.transform(nombres_base)
    similitud = cosine_similarity(titulo_vec, target_vec)
    return similitud[0]


# Función de tipo sigmoide para asignar puntuaciones de similitud en función de la distancia desde el precio de referencia y entre 0 y 1.
#Necesita mejor calibración de seguir usandose, tal vez otra fucnión una aproximación más lineal a partir de la diferencia de precios.
def precio_similitud(diferencia_precio):
    factor = 0.0005
    if diferencia_precio == 0:
        return 1.0
    similitud = 1 / (1 + np.exp(-factor * diferencia_precio))
    return round(similitud, 3)



#Función que calcula un ponderado de las variables de interes y un peso dado para etsimar al mejor candidato
def ponderado_productos_similares(df, n):
    #Asignación de pesos a las variables, agergar más para mejores resultados (Marca, estado, tags)
    peso_title = 0.7
    peso_precio = 0.3

    df['sumatoria_pesos'] = ( peso_title* df['name_similarity']) + (peso_precio * df['price_similarity'])
    
    
    # Sort the DataFrame by the weighted sum in descending order
    data = df.sort_values(by='sumatoria_pesos', ascending=False)
    
    # Get the top n products with the highest scores
    top_n_products = data.head(n)
    
    return top_n_products



In [92]:
def match_productos(mla_id_in):
    data_instance = data_inicial() 
    cat_interes = data_instance.extrae_cat_objetivo(mla_id_in)
    base_objetivo = data_instance.base_inicial(cat_interes[1])
    precio_objetivo = cat_interes[2]
    #Eliminar misma publicación de la base objetivo
    base_objetivo = base_objetivo.drop(base_objetivo[base_objetivo['MLA_id'] == mla_id_in].index)
    titulo_clean = data_instance.clean_text(cat_interes[0])
    base_nombres = data_instance.busca_titulos(base_objetivo)
    name_match_score = similitud_nombre(titulo_clean,base_nombres,base_objetivo)
    base_objetivo['name_similarity'] = base_objetivo.apply(lambda row: name_match_score[row.name], axis=1)
    base_objetivo['price_similarity'] = base_objetivo.apply(lambda row: precio_similitud(row['price'] - precio_objetivo), axis=1)
    return base_objetivo

Cambiar el id del producto sobre el cual queremos encontrar sugerencias similares


<h1>Solución: </h1>

1. **Obtén el `MLA_id` del Producto de Interés:** Identifica el producto para el cual deseas encontrar productos similares. 

2. **Modifica la variable 'id_busqueda', aquí tienes algunos ejemplos:
    -**MLA1388506360
    -**MLA1229972956
3. **Ejecuta la siguientes celdas para obetener el resultado
    

In [98]:
id_busqueda = 'MLA1388506360'
base_resultado = match_productos(id_busqueda)

New model created and saved successfully.


In [100]:
data_instance = data_inicial() 
objeto_inicial = data_instance.extrae_cat_objetivo(id_busqueda)
print(f'Producto inicial MLA_id: ',id_busqueda)
print(f'Producto inicial titulo: ',objeto_inicial[0])
print(f'Producto inicial ML: ',objeto_inicial[1])
print(f'Producto inicial precio: ',objeto_inicial[2])

top_5_products = ponderado_productos_similares(a, n=5)

top_5_products.head()

Producto inicial MLA_id:  MLA1388506360
Producto inicial titulo:  Monitor Led 24 Pulgadas Philips 241v8l/77 Hdmi Vga
Producto inicial ML:  MLA14407
Producto inicial precio:  81199


,MLA_id,title,condition,permalink,category_id,domain_id,tags,price,brand,model,thumbnail,url,cleaned_title,name_similarity,price_similarity,sumatoria_pesos
3,MLA1364919355,Monitor Philips Led 24 Pulgadas 241v8l/77 Hdmi...,new,https://www.mercadolibre.com.ar/monitor-philip...,MLA14407,MLA-COMPUTER_MONITORS,"[extended_warranty_eligible, cart_eligible, be...",57999.00,Philips,241V8L/77,http://http2.mlstatic.com/D_747044-MLA51839578...,https://http2.mlstatic.com/D_747044-MLA5183957...,monitor philip led 24 pulgadas 241v8l77 hdmi vga,0.918883,0.000,0.643218
22,MLA1130582604,Monitor Philips V 272v8la/55 Lcd 27 Negro 100...,new,https://www.mercadolibre.com.ar/monitor-philip...,MLA14407,MLA-COMPUTER_MONITORS,"[extended_warranty_eligible, good_quality_pict...",87999.00,Philips,272V8LA/55,http://http2.mlstatic.com/D_879503-MLA52231599...,https://http2.mlstatic.com/D_879503-MLA5223159...,monitor philip v 272v8la55 lcd 27 negro 100v240v,0.252292,0.968,0.467004
25,MLA1457260576,Monitor Philips Pc De 27'' Full Hd - 271e1sca/...,new,https://www.mercadolibre.com.ar/monitor-philip...,MLA14407,MLA-COMPUTER_MONITORS,"[ahora-paid-by-buyer, deal_of_the_day, extende...",129999.00,Philips,271E1SCA/55,http://http2.mlstatic.com/D_990408-MLA52364993...,https://http2.mlstatic.com/D_990408-MLA5236499...,monitor philip pc 27 full hd 271e1sca55 color ...,0.206308,1.000,0.444416
33,MLA1121584547,Monitor Gamer Asus Gaming Vg248qg Led 24 Negr...,new,https://www.mercadolibre.com.ar/monitor-gamer-...,MLA14407,MLA-COMPUTER_MONITORS,"[extended_warranty_eligible, good_quality_pict...",161490.64,Asus,VG248QG,http://http2.mlstatic.com/D_887397-MLA46544970...,https://http2.mlstatic.com/D_887397-MLA4654497...,monitor gamer asus gaming vg248qg led 24 negro...,0.189053,1.000,0.432337
28,MLA1381672003,Monitor Gamer LG Ultragear 24gn600 Led 24 Neg...,new,https://www.mercadolibre.com.ar/monitor-gamer-...,MLA14407,MLA-COMPUTER_MONITORS,"[deal_of_the_day, extended_warranty_eligible, ...",181999.00,LG,24GN600,http://http2.mlstatic.com/D_845070-MLA46623210...,https://http2.mlstatic.com/D_845070-MLA4662321...,monitor gamer lg ultragear 24gn600 led 24 negr...,0.172801,1.000,0.420960
